In [1]:
import pandas as pd
from itertools import product

In [2]:
import inspect, re

def varname(p):
  for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
    m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
    if m:
      return m.group(1)

In [3]:
df = pd.read_excel('TestList.xlsx')
segments = sorted(list(set(df.Segment.to_list())))
segments

['ChunkStorage',
 'ChunkingAlgorithm',
 'Embedder',
 'FAISS Vector',
 'Query',
 'Reranker',
 'TextNormalize']

### ChunkStorage

In [4]:
scoring_function = ['BM25F']
K1 = [1, 1.2, 1.5, 2.0]
B = [0.5, 0.75, 1, 1.25]

WhooshChunkStorage = [
    {
        'scoring_function': sf,
        'K1': k1,
        'B': b
    }
    for sf, k1, b in product(scoring_function, K1, B)
]
ChunkStorage = [{
    'method': varname(WhooshChunkStorage),
    'params': i
    } for i in WhooshChunkStorage]

### ChunkingAlgorithm

Word count

In [5]:
word_amount = [128, 256, 512, 1024, 2048]
overlay = [0.1, 0.25, 0.5]

WordCountingChunkingAlgorithm = [
    {
        'words_amount': wa,
        'overlap': o
    }
    for wa, o in product(word_amount, overlay) 
]

Sentence count

In [6]:
sentence_amount = [2, 4, 8, 16, 32]
overlay = [0.1, 0.25, 0.5]

SentenceChunkingAlgorithm = [
    {
        'sentences_in_chunk': sa,
        'overlap': o
    }
    for sa, o in product(sentence_amount, overlay)
]

In [7]:
ChunkingAlgorithm = [{
    'method': varname(WordCountingChunkingAlgorithm),
    'params': i
    } for i in WordCountingChunkingAlgorithm] + [{
        
    'method': varname(SentenceChunkingAlgorithm),
    'params': i
    } for i in SentenceChunkingAlgorithm]


### Embedder

In [8]:
# model = ["BAAI/bge-m3", 'SFR-Embedding-Mistral', 'mxbai-embed-large-v1']
model = ['intfloat/e5-small-v2', "BAAI/bge-m3", 'thenlper/gte-base', 'DeepPavlov/rubert-base-cased-sentence']
HuggingFaceEmbedder = [{'model': i} for i in model]

In [9]:
Embedder = [{     
    'method': varname(HuggingFaceEmbedder),
    'params': i
    } for i in HuggingFaceEmbedder]
            

### FAISS Vector

In [10]:
FAISS_Vector = [{
    'method': 'FaissIndexFlatL2',
    'params': None}] +[{
        
    'method': 'FaissIndexFlatIP',
    'params': None}]

### LLM

In [11]:
model = ['7b', '8b', '12b']
LLM = [{'model': i} for i in model]
LLM = [{
    'method': varname(LLM),
    'params': i
    } for i in LLM]

### Create final tets file

In [12]:
test_combinations = [{
    # varname(ChunkStorage): i[0],
    varname(ChunkingAlgorithm): i[0],
    varname(Embedder): i[1],
    # varname(FAISS_Vector): i[2],
    # varname(LLM): i[3]
}
for i in product( ChunkingAlgorithm, Embedder)]

In [13]:
len(test_combinations)

120

In [14]:
import json
with open('test_combinations.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(test_combinations, indent=4))

In [15]:
import sys
sys.path.append('..')

In [16]:
import mirage
import inspect

print(type(WordCountingChunkingAlgorithm.__qualname__))
class_translator: dict[str, type] = {
    name : cls for name, cls in inspect.getmembers(
        sys.modules['mirage']
    ) if inspect.isclass(cls)
}


ModuleNotFoundError: No module named 'pydantic'

In [ ]:
sys.modules['mirage']

<module 'mirage' from 'c:\\Users\\Bulkin\\MIRAGE\\generate_final_test_desc\\..\\mirage\\__init__.py'>

In [ ]:
class_translator

{'ABC': abc.ABC,
 'Any': typing.Any,
 'BaseModel': pydantic.main.BaseModel,
 'BaselineIndex': mirage.index.BaselineIndex.BaselineIndex,
 'BowEmbedder': mirage.embedders.BowEmbedder.BowEmbedder,
 'ChunkNote': mirage.index.chunk_storages.ChunkStorage.ChunkNote,
 'ChunkStorage': mirage.index.chunk_storages.ChunkStorage.ChunkStorage,
 'ChunkingAlgorithm': mirage.index.chunking_algorithms.ChunkingAlgorithm.ChunkingAlgorithm,
 'Embedder': mirage.embedders.Embedder.Embedder,
 'EmbedderIsNotTrainedException': mirage.embedders.Embedder.EmbedderIsNotTrainedException,
 'FaissIndexFlatIP': mirage.index.vector_index.FaissVectorIndex.FaissIndexFlatIP,
 'FaissIndexFlatL2': mirage.index.vector_index.FaissVectorIndex.FaissIndexFlatL2,
 'FaissIndexHNSWFlat': mirage.index.vector_index.FaissVectorIndex.FaissIndexHNSWFlat,
 'FaissIndexIVFFlat': mirage.index.vector_index.FaissVectorIndex.FaissIndexIVFFlat,
 'FaissIndexIVFPQ': mirage.index.vector_index.FaissVectorIndex.FaissIndexIVFPQ,
 'FaissIndexIVFPQR': m

In [ ]:
chunking_cls = class_translator[a[0]['ChunkingAlgorithm']['method']]
chunking_cls(**a[0]['ChunkingAlgorithm']['params'])


TypeError: WordCountingChunkingAlgorithm.__init__() got an unexpected keyword argument 'word_amount'

In [ ]:
inspect.getmembers(
        sys.modules['mirage']
    )

[('ABC', abc.ABC),
 ('Any', typing.Any),
 ('BaseModel', pydantic.main.BaseModel),
 ('BaselineIndex', mirage.index.BaselineIndex.BaselineIndex),
 ('BowEmbedder', mirage.embedders.BowEmbedder.BowEmbedder),
 ('Callable', typing.Callable),
 ('ChunkNote', mirage.index.chunk_storages.ChunkStorage.ChunkNote),
 ('ChunkStorage', mirage.index.chunk_storages.ChunkStorage.ChunkStorage),
 ('ChunkingAlgorithm',
  mirage.index.chunking_algorithms.ChunkingAlgorithm.ChunkingAlgorithm),
 ('Dict', typing.Dict),
 ('Embedder', mirage.embedders.Embedder.Embedder),
 ('EmbedderIsNotTrainedException',
  mirage.embedders.Embedder.EmbedderIsNotTrainedException),
 ('FaissIndexFlatIP',
  mirage.index.vector_index.FaissVectorIndex.FaissIndexFlatIP),
 ('FaissIndexFlatL2',
  mirage.index.vector_index.FaissVectorIndex.FaissIndexFlatL2),
 ('FaissIndexHNSWFlat',
  mirage.index.vector_index.FaissVectorIndex.FaissIndexHNSWFlat),
 ('FaissIndexIVFFlat',
  mirage.index.vector_index.FaissVectorIndex.FaissIndexIVFFlat),
 ('Fai